# Analyzing CTCAC tax credits project applications from previous years 
Context: 2025 4% allocations. After training on round 1 data, the model is used to predict the round 2 (I did this manually), but only achieved 60% accuracy (terrible result)  
Testing on round 2 data shows that our model trained on only round 1 data is insufficient.   
Possible solutions:
- Get more training data from previous years (get at least 1000 data points if possible)
- include fund allocation per round data into the training 



In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

# ensure changes in files are reflected
%load_ext autoreload
%autoreload 2
# Set display options to show all columns
pd.set_option('display.max_columns', None)  # Show all columns

In [ ]:
R1_2024_applicant = pd.read_excel("../data/external/2024-R1-ApplicantList.xlsx", header=1, index_col=None)
R2_2024_applicant = pd.read_excel("../data/external/2024-R2-ApplicantList.xlsx", header=1, index_col=None)
award_2024 = pd.read_excel("../data/external/2024-Financing-data.xlsx", index_col=None)

R1_2023_applicant = pd.read_excel("../data/external/2023-R1-ApplicantList.xlsx", header=1, index_col=None)
R2_2023_applicant = pd.read_excel("../data/external/2023-R2-ApplicantList.xlsx", header=1, index_col=None)
R3_2023_applicant = pd.read_excel("../data/external/2023-R3-ApplicantList.xlsx", header=1, index_col=None)
award_2023 = pd.read_excel("../data/external/2023-Financing-data.xlsx", index_col=None)

df_list = [R1_2024_applicant, R2_2024_applicant, award_2024, 
            R1_2023_applicant, R2_2023_applicant, R2_2023_applicant, award_2023]

In [31]:
for df in df_list: 
    print(df.columns)
    print(df.shape)

Index(['APPLICATION NUMBER', 'PROJECT NAME', 'CONSTRUCTION TYPE',
       'HOUSING TYPE', 'CITY', 'COUNTY', 'TOTAL UNITS', 'LOW INCOME UNITS',
       'MARKET RATE UNITS', 'UNITS FOR HOMELESS INDIVIDUALS',
       'AVERAGE AFFORDABILTY (TARGETED AMI)', 'TOTAL PROJECT COST',
       'CONSTRUCTION BOND TAX-EXEMPT FINANCING AMOUNT',
       'ANNUAL FEDERAL CREDIT REQUESTED', 'TOTAL STATE CREDIT REQUESTED',
       'CDLAC NON-GEPGRAPHIC POOL', 'NEW CONSTRUCTION SET ASIDE',
       'SECONDARY NEW CONSTRUCTION SET ASIDE IF APPLICABLE',
       'CTCAC GEOGRAPHIC REGION', 'CDLAC GEOGRAPHIC REGION',
       'CDLAC TOTAL POINTS SCORE',
       'PRESERVATION AND OTHER REHAB. PROJECT PRIORITIES (20 PTS)',
       'NEW CONSTRUCTION DENSITY & LOCAL INCENTIVES (10 PTS)',
       'EXCEEDING MINIMUM INCOME RESTRICTIONS (20 PTS)',
       'EXCEEDING MINIMUM RENT RESTRICTIONS (10 PTS)',
       'GP & MGMT. CO. EXPERIENCE (10 PTS)', 'HOUSING NEEDS (10 PTS)',
       'LEVERAGED SOFT RESOURCES (8 PTS)', 'READINESS TO PROC

## Issue
The columns and their names are all slightly different from one another. How do I merge them into a single file? What about future excel files? Is it possible to setup a pipeline that deals with all these variability? 

In [41]:
import re

for df in df_list: 
    print(df.shape)
    cols_to_drop = [col for col in df.columns if (re.match(r'GP.*', col) or re.match(r'(.* PTS)', col) or  re.match(r'Points,.*', col))]
    print(cols_to_drop)
    df = df.drop(columns=cols_to_drop)
    print("new df shape: ", df.shape)
    print(df.columns)
    print("-----------------\n")

(161, 45)
['PRESERVATION AND OTHER REHAB. PROJECT PRIORITIES (20 PTS)', 'NEW CONSTRUCTION DENSITY & LOCAL INCENTIVES (10 PTS)', 'EXCEEDING MINIMUM INCOME RESTRICTIONS (20 PTS)', 'EXCEEDING MINIMUM RENT RESTRICTIONS (10 PTS)', 'GP & MGMT. CO. EXPERIENCE (10 PTS)', 'HOUSING NEEDS (10 PTS)', 'LEVERAGED SOFT RESOURCES (8 PTS)', 'READINESS TO PROCEED (10 PTS)', 'AFFIRMATIVELY FURTHERING FAIR HOUSING (10 PTS)', 'SERVICE AMENITIES (10 PTS)', 'COST CONTAINMENT (12 PTS)', 'SITE AMENITIES (10 PTS)', 'GP 1 COMPANY NAME', 'GP 1 CONTACT NAME', 'GP 1 PARENT COMPANY', 'GP 2 COMPANY NAME', 'GP 2 CONTACT NAME', 'GP 2 PARENT COMPANY', 'GP 3 COMPANY NAME', 'GP 3 CONTACT NAME', 'GP 3 PARENT COMPANY']
new df shape:  (161, 24)
Index(['APPLICATION NUMBER', 'PROJECT NAME', 'CONSTRUCTION TYPE',
       'HOUSING TYPE', 'CITY', 'COUNTY', 'TOTAL UNITS', 'LOW INCOME UNITS',
       'MARKET RATE UNITS', 'UNITS FOR HOMELESS INDIVIDUALS',
       'AVERAGE AFFORDABILTY (TARGETED AMI)', 'TOTAL PROJECT COST',
       'CONST